In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as pl
import os
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization, Activation, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, LeakyReLU, concatenate
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow import clip_by_value
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
import pickle
from random import sample
import gc

In [ ]:
train_images = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Images\\train_images.parquet').read().to_pandas()
test_images = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Images\\test_images.parquet').read().to_pandas()
val_images = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Images\\val_images.parquet').read().to_pandas()

train_labels = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Labels\\train_labels.parquet').read().to_pandas()
test_labels = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Labels\\test_labels.parquet').read().to_pandas()
val_labels = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Labels\\val_labels.parquet').read().to_pandas()

train_images = np.array(train_images).reshape(train_images.shape[0],48,48,1)
test_images = np.array(test_images).reshape(test_images.shape[0],48,48,1)
val_images = np.array(val_images).reshape(val_images.shape[0],48,48,1)

In [ ]:
leaky_alphas = [alpha/1000 for alpha in range(1, 52, 5)]
learning_rates = [learning_rate/1000000 for learning_rate in range(50,160,10)]

In [ ]:
param_configs = []
for leaky_alpha in leaky_alphas:
    for learning_rate in learning_rates:
        param_configs.append((leaky_alpha, learning_rate))

In [ ]:
sample_count = 50

In [ ]:
sampled_param_configs = sample(param_configs, sample_count)

In [ ]:
def network_train(param_config: tuple, param_results: dict, epochs: int):
    print(f'Leaky Alpha: {param_config[0]}, Learning Rate: {param_config[1]}')
    cnn_input = Input(shape=(48,48,1))
    x = Conv2D(224, kernel_size=6, activation=LeakyReLU(alpha=param_config[0]))(cnn_input)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.5)(x)
    x = Conv2D(448, kernel_size=2, activation=LeakyReLU(alpha=param_config[0]))(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    cnn_output = Dense(7, activation='softmax')(x)
    
    cnn = Model(inputs=[cnn_input], outputs=cnn_output)
    optimizer = optimizers.Adam(learning_rate=param_config[1])
    cnn.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    cnn_history = cnn.fit(train_images, train_labels, validation_data=(test_images, test_labels),
                          batch_size=200, epochs=epochs)
    
    val_accuracy = cnn_history.history['val_accuracy'][-1]
    avg_accuracy_diff = np.mean(np.array(cnn_history.history['val_accuracy'][epochs-6:epochs])-np.array(cnn_history.history['accuracy'][epochs-6:epochs]))
    param_results[param_config] = (val_accuracy, avg_accuracy_diff)
    
    del cnn_input
    del cnn_output
    del cnn
    gc.collect()
    keras.backend.clear_session()
    print(f'Average Accuracy Difference: {round(avg_accuracy_diff,3)}\n')

In [ ]:
param_results = {}

In [ ]:
for param_config in sampled_param_configs:
    network_train(param_config, param_results, epochs=12)

In [ ]:
leaky_alphas = []
learning_rates = []
max_val_accuracies = []
avg_val_differences = []
for param_result in sampled_param_configs:
    leaky_alphas.append(param_result[0])
    learning_rates.append(param_result[1])
    max_val_accuracies.append(param_results[param_result][0])
    avg_val_differences.append(param_results[param_result][1])

In [ ]:
leaky_alphas, learning_rates, max_val_accuracies, avg_val_differences = pd.DataFrame(leaky_alphas).rename(columns={0:'leaky_alpha'}), pd.DataFrame(learning_rates).rename(columns={0:'learning_rate'}), pd.DataFrame(max_val_accuracies).rename(columns={0:'max_val_accuracy'}), pd.DataFrame(avg_val_differences).rename(columns={0:'avg_val_difference'})

In [ ]:
plot_df = pd.concat([leaky_alphas, learning_rates, max_val_accuracies, avg_val_differences], axis=1)

In [ ]:
hyperparameter_scaler = StandardScaler()

In [ ]:
plot_df[['max_val_accuracy', 'avg_val_difference']] = pd.DataFrame(hyperparameter_scaler.fit_transform(plot_df[['max_val_accuracy', 'avg_val_difference']]))

In [ ]:
pl.scatter(plot_df['leaky_alpha']*100, plot_df['learning_rate']*10000,
           c=plot_df['max_val_accuracy'],
           s=plot_df['avg_val_difference']*500)
pl.legend()
pl.colorbar()
pl.show()

In [ ]:
leaky_alpha = 2.1/100
learning_rate = 1.25/10000

In [ ]:
print(f'Tuned training parameters: learning_rate = {learning_rate}, leaky_alpha = {leaky_alpha}')